In [ ]:
%pip install requests
%pip install scikit-learn
%pip install python-dotenv

In [ ]:
# Trials Notebook: Experiment with fetching news and personalization

import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
# Set up News API key
NEWS_API_KEY = os.getenv('NEWS_API_KEY')

# Fetch news articles based on a keyword
def fetch_articles_by_keyword(keyword, page_size=5):
    url = f"https://newsapi.org/v2/everything?q={keyword}&pageSize={page_size}&apiKey={NEWS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get('articles', [])
    else:
        print("Error fetching articles:", response.status_code)
        return []

In [ ]:
# Experiment with recommendations
def recommend_articles(user_keywords, articles):
    article_texts = [article['title'] + " " + article['description'] for article in articles]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(user_keywords + article_texts)
    similarities = cosine_similarity(tfidf_matrix[0:len(user_keywords)], tfidf_matrix[len(user_keywords):])
    recommendations = sorted(list(enumerate(similarities.sum(axis=0))), key=lambda x: x[1], reverse=True)
    return [articles[i[0]] for i in recommendations[:5]]

In [ ]:
# Example trial
if __name__ == "__main__":
    keyword = "AI"
    user_keywords = ["Artificial Intelligence", "Machine Learning"]
    articles = fetch_articles_by_keyword(keyword)
    recommended_articles = recommend_articles(user_keywords, articles)

    for idx, article in enumerate(recommended_articles, start=1):
        print(f"{idx}. {article['title']}")
        print(article['description'])
        print(article['url'])
        print("-" * 80)